In [1]:
!pip install transformers
!pip install torch
!pip install -U datasets

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Convert the dataset text to a csv:

In [3]:
text_filepath = "../home/schen9/dataset.txt"
csv_filepath = "dataset.csv"

In [11]:
from itertools import islice

with open(text_filepath) as input_file:
    head = list(islice(input_file, 10))
print(head)

['The White Monkey is a 1925 American silent drama film, directed by Phil Rosen and starring Barbara La Marr, Thomas Holding, and Henry Victor.\n', 'It was released by First National Pictures on June 7, 1925.\n', 'Plot\n', 'As described in a film magazine review, Fleur, daughter of Soames Forsyte, marries Michael Mont, whose best friend Wilfrid Desert, author, painter, and traveler, is also in love with her.\n', 'He knows that Fleur married Michael without any love for him.\n', 'When Fleur refuses to allow him to leave London, he decides that he has a chance to win her.\n', 'Wilfrid tells Michael that he will win her away from him if he can.\n', 'Michael, also of the generation after the war, refuses to coerce Fleur or attempt any heroics with Wilfred.\n', 'He later discovers them together after she has told him that she has gone.\n', 'Cast list\n']


In [5]:
with open(text_filepath, "r", encoding="utf-8") as text_file:
    rows = text_file.read().strip().split("\n\n")

In [7]:
import csv

with open(csv_filepath, "w", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Text"])
    for i, row in enumerate(tqdm(rows)):
        writer.writerow([row.strip()])

print(f"Finished writing to {csv_filepath}") 

  0%|          | 0/449919 [00:00<?, ?it/s]

Finished writing to dataset.csv


In [72]:

type("abcdefgh00ijklm".split("0")) == str
type("dd") == str

True

In [8]:
from datasets import Dataset, load_dataset

dataset = load_dataset("csv",data_files=csv_filepath) 
dataset = dataset.filter(lambda x: x["Text"] is not None) # filter out NoneTypes
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/449919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 446203
    })
})

In [87]:
print(type(dataset["train"]["Text"][11]))
dataset["train"]["Text"][11]

<class 'str'>


"Grasslands\nMontane grasslands and shrublands\nTemperate grasslands, savannas, and shrublands\nEcoregions\nPlains\nPrairies\nNushi Tulliu (23 April 1872 – 8 April 1941; ) was an Aromanian poet and prose writer.\nHe was born into an Aromanian family in Avdella, a village that formed part of the Ottoman Empire's Manastir Vilayet and is now in Greece.\nHe began school in Kleisoura, followed by the Romanian high school in Bitola.\nTulliu then enrolled in the literature faculty at the University of Bucharest in the Romanian Old Kingdom, where he took a degree in Romanian history and language.\nHe attended the law faculty for two years, but did not graduate.\nIn 1905, he was sent to Leipzig University in order to prepare his doctorate in history.\nWhile still a student, he taught at the Romanian gymnasium in Ioannina, and then in Romania.\nFrom 1929 until his retirement in 1937, he was a teacher in Bucharest.\nFrom 1908 to 1912, he was a school inspector in Macedonia.\nTulliu's first publis

In [ ]:
input_ids = torch.tensor(tokenizer.encode("Hello, my <span class="highlighter highlight-on">dog</span> is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)

last_hidden_states = outputs[0] #(batch_size, input_len, embedding_size) But I need single vector for each sentence 

In [9]:
# df = pd.read_csv(csv_filepath)
# df
df = dataset["train"].to_pandas()
df

,Text
0,The White Monkey is a 1925 American silent dra...
1,Preservation\nAn incomplete print of The White...
2,External links
3,Films based on works by John Galsworthy\nFilms...
4,the montane grasslands and shrublands biome\n ...
...,...
446198,References
446199,External links
446200,"1929 births\n2015 deaths\nPeople from Naseby, ..."
446201,Scottish female golfers\nGolfers from Edinburg...


In [6]:
strings = df["Text"].tolist()
strings

['The White Monkey is a 1925 American silent drama film, directed by Phil Rosen and starring Barbara La Marr, Thomas Holding, and Henry Victor.\nIt was released by First National Pictures on June 7, 1925.\nPlot\nAs described in a film magazine review, Fleur, daughter of Soames Forsyte, marries Michael Mont, whose best friend Wilfrid Desert, author, painter, and traveler, is also in love with her.\nHe knows that Fleur married Michael without any love for him.\nWhen Fleur refuses to allow him to leave London, he decides that he has a chance to win her.\nWilfrid tells Michael that he will win her away from him if he can.\nMichael, also of the generation after the war, refuses to coerce Fleur or attempt any heroics with Wilfred.\nHe later discovers them together after she has told him that she has gone.\nCast list',
 'Preservation\nAn incomplete print of The White Monkey is located at the Library of Congress and the Wisconsin Center for Film and Theater Research.\nThe surviving material co

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

transformer_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

In [ ]:
tokens = []
for i, row in enumerate(tqdm(df["Text"])):
    tokens.append([tokenizer(row)])

  0%|          | 0/446203 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df["Tokens"] = df.apply(lambda row: tokenizer(row['Text']), axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors


In [29]:
tokenizer.encode(["helll yeahh I want to finish assignment 4"])

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [8]:
tokens = tokenizer(strings, padding="max_length", truncation=True)
tokens

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [10]:
dataset["train"][0]

{'Text': 'The White Monkey is a 1925 American silent drama film, directed by Phil Rosen and starring Barbara La Marr, Thomas Holding, and Henry Victor.\nIt was released by First National Pictures on June 7, 1925.\nPlot\nAs described in a film magazine review, Fleur, daughter of Soames Forsyte, marries Michael Mont, whose best friend Wilfrid Desert, author, painter, and traveler, is also in love with her.\nHe knows that Fleur married Michael without any love for him.\nWhen Fleur refuses to allow him to leave London, he decides that he has a chance to win her.\nWilfrid tells Michael that he will win her away from him if he can.\nMichael, also of the generation after the war, refuses to coerce Fleur or attempt any heroics with Wilfred.\nHe later discovers them together after she has told him that she has gone.\nCast list'}

In [14]:
def tokenize(batch):
    text = "".join(batch["Text"])
    return tokenizer(batch["Text"], truncation=True)

batch_size = 20
tokens = dataset.map(tokenize, batched=True, batch_size=batch_size)
tokens

Map:   0%|          | 0/446203 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text', 'input_ids', 'attention_mask'],
        num_rows: 446203
    })
})

In [17]:
l = ["yo","1","2"]
t = "".join(l)
t
print(type(l))

<class 'list'>


In [6]:
# Read the giant textfile in
filepath = "../home/schen9/dataset.txt"

def read_file(filepath,num_lines=850_000):
    # text_lines = []
    text = ""
    with open(filepath) as fh:
        for i, line in enumerate(fh):
            if i >= num_lines:
                break
            # text_lines.append(line)
            text += line
    # return text_lines
    return text

text = read_file(filepath)

In [ ]:
tokens = tokenizer(text, truncation=True, return_tensors='pt')

In [6]:
tokens

{'input_ids': tensor([[    0,   133,   735, 34546,    16,    10, 36248,   470,  8454,  4149,
           822,     6,  3660,    30,  4720, 11998,     8,  8996,  4810,  1587,
          1127,   338,     6,  1813, 11549,     6,     8,  4858,  8804,     4,
         50118,   243,    21,   703,    30,  1234,   496, 12290,    15,   502,
           262,     6, 36248,     4, 50118, 48812, 50118,  1620,  1602,    11,
            10,   822,  4320,  1551,     6, 12602,   710,     6,  1354,     9,
           407, 12336, 18095,   219,   859,     6,  4401,  4458,   988,  4436,
             6,  1060,   275,  1441,  3884,   506, 10505, 15071,     6,  2730,
             6, 25760,     6,     8, 33489,     6,    16,    67,    11,   657,
            19,    69,     4, 50118,   894,  2215,    14, 12602,   710,  2997,
           988,   396,   143,   657,    13,   123,     4, 50118,  1779, 12602,
           710, 16766,     7,  1157,   123,     7,   989,   928,     6,    37,
         11703,    14,    37,    34,  

In [8]:
model(**tokens)

MaskedLMOutput(loss=None, logits=tensor([[[37.5077, -3.9347, 15.3525,  ...,  3.8584,  5.4767, 12.8930],
         [ 7.3782, -2.7824, 13.6218,  ...,  4.8712,  5.0904,  8.2832],
         [ 1.1484, -3.6915,  7.2563,  ..., -0.1078, -1.0249,  3.4536],
         ...,
         [ 0.2669, -4.2053,  7.0668,  ..., -5.6054, -5.8515,  2.2009],
         [-1.2186, -3.4854, 15.0709,  ...,  2.3310, -3.1703,  4.0132],
         [ 8.2976, -3.9727, 28.2076,  ...,  1.5808, -2.9764,  8.9007]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)